In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction to Natural Language Processing

### Python packages used

In [ ]:
## Packages for data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Packages for basic text processing
import re
import string 

## Import Logistic Regression function for model training from linear model module of sklearn package
from sklearn.linear_model import LogisticRegression

## Import functions for evaluation from  model selection module of sklearn package
from sklearn.metrics import confusion_matrix, accuracy_score

## Import functions for model selection from  model selection module of sklearn package
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

## Import functions for text vectorization from feature extraction module of sklearn package
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Import function from NLTK package for Text tokenization and Normalization
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

## Import matplotlib and wordcloud for Visualization
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

## Import IFrame for display videos
from IPython.display import IFrame

## Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Natural Language Processing

#### What is Natural Language?

    What does it mean, Natural Language is which we humans use to communicate our messages,views and expressions.

#### Is there any non Natural Language?

    We can say that all programming language low or high level are non Natural Language because there structure is well defined and they are built for systems to communicate within themselves or for humans to express the logic for computers.

#### Which one is complicated for systems to process?

    Because programming language are designed for systems, even they might be difficult for humans to learn but for systems they are well strucuted and not complicated to process but human languages are not designed or developed to have this in mind. Human or Natural language needs to process and transalated into the language or representation which systems can understand. Natural language is result of human evalution and resulted from need to communicate the messages over the time, And humans just not use the words but there are many other aspects like expressions, tone of voice and how we structure the same words in different sentences to convey different meanings. For that reason it makes Natural Language a lot difficult to process and interpret underlying messages for systems which might be simple and straightforward for humans in most of the cases. Other than that humans do not have single language for communication globally and that makes it even more complicated because there is no single underlying structure.

    Natural Language Processing (NLP) is a field which deals with processing of Natural Languages for systems so that interpretation of these languages can be used in many other applications. There are further sub fields of NLP which are Natural Language Understanding (NLU) and Natural Language Generation (NLG).

    Natural Language Processing is sub field of Artificial Intelligence and deals the part of AI where there is need to process Natural Language to make intelligent agents.
    
    Natural Language Processing is not a single field but it combines multiple fields like Linguistics, Computer Science and Machine Learning
    
#### Applications of Natural Language Processing

1. Machine Translation
2. Question Answering
3. Chatbots
4. Summarizatin
5. Information Extraction

Watch the followng video from Dan Jurafsky for introduction to Natural Language Processing:

In [ ]:
IFrame("https://www.youtube.com/embed/oWsMIW-5xUc?controls=0",width=600,height=400)

## Predicting Disaster Tweets

For this tutorial we will use dataset from Kaggle playground competition "Natural Language Processing with Disaster Tweets". Objective of this problem is to predicts which Tweets are about real disasters and which one’s aren’t. Dataset contains tweets that were hand classified as disaster or non disaster. For more information you can visit following link:

https://www.kaggle.com/c/nlp-getting-started

We will use Supervised Machine Learning technique Logistic Regression for building prediction model. To train Logistic model we will use linear models package from Sklearn machine learning python package, for documentation look into the below documentation for logistic regression model:
    
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
  
To learn more about Logistic Regression watch the following video from Andrew Ng's Machine Learning course:

In [ ]:
IFrame('https://www.youtube.com/embed/-la3q9d7AKQ?controls=0',width=600,height=400)

## 1. Load data and basic data exploration

In [ ]:
tweets_dataset = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

### 1.1 Load Data

#### 1.1.1 Quick look at tweets dataset

Load data using pandas package from csv file and explore

- How many rows and columns are there in dataset?
- Quick look at first few rows of dataset

In [ ]:
print("Tweets dataset contains %d rows and %d columns." % tweets_dataset.shape)
tweets_dataset.head()

#### 1.1.2 Keep only text and target columns for further processing

For this tutorial, we need text column and target labels, so will discard any other columns from dataset

In [ ]:
tweets_dataset=tweets_dataset[['text','target']]

#### 1.1.3 Look the distribution of target variable

It is important to look at the distribution of target variable, because metrics used later for evaluation might be misleading if target value has imbalanced distribution of True and False labels

In [ ]:
tweets_dataset.target.value_counts().sort_values().plot(kind = 'barh')

##### Target value 1 means, the text is labeled as disaster tweet

In [ ]:
print("%.1f tweets are labeled as disaster tweets in data" % (
    (tweets_dataset.target[tweets_dataset.target==1].count()/tweets_dataset.target.count())*100))

### 1.2 Split dataset into train and test datasets

For predicting disaster tweets using NLP problem, we will first split the data into two groups one for building or training the model and other set to evaluate the prediction accuracy on unseen data. For more about splitting the data into training and test please go through below link

https://developers.google.com/machine-learning/crash-course/training-and-test-sets/splitting-data

#### 1.2.1 Split the dataset into 70:30 ratio. 70% for training the model and 30% for validation

In [ ]:
train, test = train_test_split(tweets_dataset,test_size=0.3,random_state=1)

#### 2.2 Explore the distribution of train and test datasets

In [ ]:
print("Train dataset contains %d rows and %d columns." % train.shape)
print("Test dataset contains %d rows and %d columns." % test.shape)

## 2. Text Vectorization

To process the text data first we need to convert text data into numerical representation for systems to learn further from data. Vectorization is the process of converting a word ito a vector of numbers that contains the information in the word. 

- Simplest approach for vectorzing is to use counts of words.
- Other more sophisticated apprach is TF-IDF (Term Frequency-Inverse Document Frequency)
    
To learn more about these approaches go through following links:
    
1. https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
2. https://www.mygreatlearning.com/blog/bag-of-words/
    
Watch the following video from Dan Jurafsky on basic text processing and word tokenization:

In [ ]:
IFrame("https://www.youtube.com/embed/pEwBjcYdcKw?controls=0",width=600,height=400)

### 2.1 Use Bag-of-words

First approach we will use for text vectorization is Bag of words. We will use CountVectorizer() from feature_extraction module in sklearn package. For documentation go through following link:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
count_vectorizer = CountVectorizer()
train_vectors = count_vectorizer.fit_transform(train["text"])

### 2.2 Apply above learned vector transformation to test dataset

In our test data there might be some words which were not available in our vocabulary from train dataset. So we will learn the vocabulary using text from training dataset and will apply the same transformation to test data set.

In [ ]:
test_vectors = count_vectorizer.transform(test["text"])

## 3. Train a model

Next step is to train our model using training vector to learn the relationship between words in text and labels assigned for each tweet.     For traning the model we will use 5 fold cross validation. To learn about cross validation watch the following video from Josh Starmer

In [ ]:
IFrame("https://www.youtube.com/embed/fSytzGwwBVw",width=600,height=400)

We will use the F1 metric for training and selecting our model using cross_val_score() function from model_selection module of sklearn package. For more info follow the link below:

https://scikit-learn.org/stable/modules/cross_validation.html

There are several metrices which can be used while validating the model and testing the results of our predictions:

1. Confusion Matrix
2. Accuracy
2. Precision
3. Recall
4. F1 Score

For learning about these metrics go through below given links:

- https://medium.com/analytics-vidhya/confusion-matrix-accuracy-precision-recall-f1-score-ade299cf63cd
- https://www.analyticsvidhya.com/blog/2020/04/confusion-matrix-machine-learning/

In [ ]:
IFrame("https://www.youtube.com/embed/jrAyRCa7aY8",width=600,height=400)

### 3.1 Train and fit model on training dataset

First we need to initializer Logistic Regrssion model

In [ ]:
clf = LogisticRegression()

Then perform 5 fold cross validation for model selection. After model selection we need to use this model on training dataset to learn the relationshipt between text data and target labels

In [ ]:
scores = cross_val_score(clf, train_vectors, train["target"], cv=5, scoring="f1")
for k, score in zip(range(len(scores)),scores):
    print("F1 Score for fold %d is %.2f " % (k+1,score))

In [ ]:
clf.fit(train_vectors, train["target"])

## 4. Prediction

### 4.1 Predict and explore how well our model predicted the unseen test data

Now as our model has learnt some patterns using training data, now we can use it predict the labels on our remaining test dataset. Predictions can be made using predict function of Logistic Regression model.

In [ ]:
test["pred"] = clf.predict(test_vectors)

### 4.2 Let's check how well our model performed on test dataset

After predictions we need to evaluate how well our model performed on unseen data. We will look into accuracy metric and confusion matrix.

In [ ]:
print("We got %.1f%% accuracy on our test dataset" % (float(accuracy_score(test.target, test.pred))*100))

In [ ]:
confusion_matrix(test.target, test.pred)

In [ ]:
test[(test.target==1) & (test.pred==1)].shape

In [ ]:
tn, fp, fn, tp = confusion_matrix(test.target, test.pred).ravel()
tot = confusion_matrix(test.target, test.pred).sum()

print("True Negative Rate: %.1f%%" % ((tn/tot)*100))
print("False Positive Rate: %.1f%%" % ((fp/tot)*100))
print("False Negative Rate: %.1f%%" % ((fn/tot)*100))
print("True Positive Rate: %.1f%%" % ((tp/tot)*100))

## Model Summary and Next Steps

This was basic model which used simple vectorization technique and logistic regression model which provided 79.6% accuracy on unseen data. This model was built on raw text without any cleaning and processing, and there might be a lot of noise in text data. So it is very important to normalize the text data

## 5. Text Normalization

Text normalization is a pre-processing step which aims at improving the quality of the text and making it suitable to process. Below are the some basic steps for text normalization.
 
1. Case normalization: applies to the languages that use uppercase and lowercase letters. All letters are converted to the same case. Application of this step might depend on case to case basis. In some of the cases, case type of words might contain some useful information.
2. Puncuation removal: To remove the puncations in the text. In most cases, this should give a good quality.
3. Stop Word removal: Stop word removal involves removing common words such as articles (the, an) and conjuncations (and, but) among others
    - one way is to consider the frequency of occurances of words in a corpus, set of documents or text.
    - Most frequent terms can then be selected as stop words to remove. But there can be cases where frequent terms might also have significant value e.g. word economy in financial documents might be a frequent word but still captures useful information. 
    - Stop word removal helps in reducing the size of vocabulary.
4. Stemming and Lemmatization: This means reducing the word to it's root form
     - helps in reducing the vocabulary.
     - stemming and Lemmatization differs in their apprach and sophistication but serve the same objective.
     - Stemming is a a simple approach that chops off the affixes of words. Stemming is aimed at reducing vocabulary and understanding of morphological processes.
     - Lemmatization approaches this task in a more sophisticated manner, using vocabularies and morphological analysis of words. Using morphological information a words root can be returned.
     
     
Reference: Advanced Natural Language Processing with TensorFlow 2, Ashish Bansal
https://www.amazon.com/Advanced-Natural-Language-Processing-TensorFlow/dp/1800200935

In [ ]:
IFrame("https://www.youtube.com/embed/3_0rQjEgEy8?controls=0",width=600,height=400)

## 5.1 Text Cleaning and Normalization

We will perform some text cleaning and text normalization. Below are some of the operations we are going to perform on our text data:

1. Lowercase all words in text
2. Remove newline characters if any in text
3. Remove punctuations
4. Remove url's and links from text
5. Remove tags from text
6. Remove multiple spaces from text
7. Remove special characters
8. Remove stop words from text
9. Apply stemming to normalize the text

Will use word tokenizer from NLTK package for tokenization and Porter Stemmer from NLTK for stemming.

NLTK is a python package for Natural Language Processing. It is considered as Swiss Knife of NLP, it might not be good for specific task but provides a lot of functionality for many general tasks in NLP processing. Below is link for Natural Language Processing using NLTK and python if you want to learn about the package in more details:

http://www.nltk.org/book/

For text cleaning we will used regular expression and re package from python. Regular expression is very important basic tool for text processing. Below are some links to the videos from Dan Jurafsky and Christopher Manning to gain more understanding about regular expressions

In [ ]:
IFrame("https://www.youtube.com/embed/EyzTQ0OKeNw",width=600,height=400)

In [ ]:
IFrame("https://www.youtube.com/embed/sUNEGBuRWzU",width=600,height=400)

### 5.2 Part of Speech Tagging

Other important step in Text tokenization and normalization is Part-of-speech tagging. Objective of the part of speech tagging is to take a piece of text and tag each word with a Part-of-speech (POS) identifier:

Example of some POS tags:




| Tag | Meaning | English Examples
| --- | --- | --- |
|ADJ | adjective | new, good, high, special, big, local
|ADP | adposition | on, of, at, with, by, into, under
|ADV | adverb | really, already, still, early, now
|CONJ | conjunction | and, or, but, if, while, although
|DET | determiner, article | the, a, some, most, every, no, which
|NOUN | noun | year, home, costs, time, Africa
|NUM | numeral | twenty-four, fourth, 1991, 14:24
|PRT | particle | at, on, out, over per, that, up, with
|PRON | pronoun | he, their, her, its, my, I, us
|VERB | verb | is, say, told, given, playing, would

For more detailed understanding for word tokenizer and POS taggin using NLTK package refer to below link:

http://www.nltk.org/book/ch05.html

Watch the below video from Christopher Manning about Part-of-speech tagging. Though we are not going to use this for our tweet prediction but it is good to have understanding of part of speech tagging because it can be useful in other tasks.

In [ ]:
IFrame("https://www.youtube.com/embed/tJBvmkNsoN8",width=600,height=400)

In [ ]:
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
def lower_text(text):
    """
        function to convert text into lowercase
        input: text
        output: cleaned text
    """
    text = text.lower() # lowering
    return text
    
def remove_newline(text):
    """
        function to remove new line characters in text
        input: text
        output: cleaned text
    """
    text = re.sub(r'\n',' ', text)
    return text

def remove_punctuations(text):
    """
        function to remove punctuations from text
        input: text
        output: cleaned text
    """    
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text


def remove_links(text):
    """
        function to links and urls from text
        input: text
        output: cleaned text
    """    
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    
    return text

def remove_tags(text):
    """
        function to remove references and hashtags from text
        input: text
        output: cleaned text
    """    
    text = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|(\w+:\/\/\S+)","",text)
    return text
    
def remove_multiplespaces(text):
    """
        function to remove multiple spaces from text
        input: text
        output: cleaned text
    """    
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text

def remove_specialchars(text):
    """
        function to remove special characters from text
        input: text
        output: cleaned text
    """    
    text = re.sub(r'\W', ' ', text)
    return text

def remove_stopwords(text):
    """
        function to tokenize the words using nltk word tokenizer and remove the stop words using nltk package's english stop words
        input: text
        output: cleaned text
    """    
    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    return text

def word_stemming(text):
    """
        function to perform stemming using porter stemmer from nltk package
        input: text
        output: cleaned text
    """        
    text=' '.join([stemmer.stem(word) for word in word_tokenize(text)])
    return text

### 5.1.1 Below code will use above functions for text cleaning and normalization

#### 5.1.1.1 Apply the processing on Train dataset

In [ ]:
# Covert text to lowercase
train.text=train.text.apply(lambda text: lower_text(text))

# Remove newlines
train.text=train.text.apply(lambda text: remove_newline(text))

# Remove punctuations
train.text=train.text.apply(lambda text: remove_punctuations(text))

# Remove links
train.text=train.text.apply(lambda text: remove_links(text))

# Remove tags
train.text=train.text.apply(lambda text: remove_tags(text))

# Remove multiple spaces
train.text=train.text.apply(lambda text: remove_multiplespaces(text))

# Remove special characters
train.text=train.text.apply(lambda text: remove_specialchars(text))

# Remove stopwords
train.text=train.text.apply(lambda text: remove_stopwords(text))

# Apply Stemming
train.text=train.text.apply(lambda text: word_stemming(text))

#### 5.1.1.2 Apply the processing on Test dataset

In [ ]:
# Covert text to lowercase
test.text=test.text.apply(lambda text: lower_text(text))

# Remove newlines
test.text=test.text.apply(lambda text: remove_newline(text))

# Remove punctuations
test.text=test.text.apply(lambda text: remove_punctuations(text))

# Remove links
test.text=test.text.apply(lambda text: remove_links(text))

# Remove tags
test.text=test.text.apply(lambda text: remove_tags(text))

# Remove multiple spaces
test.text=test.text.apply(lambda text: remove_multiplespaces(text))

# Remove special characters
test.text=test.text.apply(lambda text: remove_specialchars(text))

# Remove stopwords
test.text=test.text.apply(lambda text: remove_stopwords(text))

# Apply Stemming
test.text=test.text.apply(lambda text: word_stemming(text))

## Text Visualization

There is a one basic visualization named as Word Clouds. This visualization provides information about frequently used words in our text corpus. wordcloud package provides word cloud visualizations in python. For more learning about wordcloud refer to the following link

https://www.analyticsvidhya.com/blog/2020/10/word-cloud-or-tag-cloud-in-python/

In [ ]:
wc_disaster = WordCloud(
        width=800, height=600,
        background_color='white',
        stopwords=STOPWORDS
    ).generate(' '.join(train[train.target==1]['text']))

wc_nondisaster = WordCloud(
        width=800, height=600,
        background_color='white',
        stopwords=STOPWORDS
    ).generate(' '.join(train[train.target==0]['text']))

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20, 12))

ax1.imshow(wc_disaster)
ax1.set_title("Word cloud of disaster tweets", fontsize=20)
ax1.axis("off")

ax2.imshow(wc_nondisaster)
ax2.set_title("Word cloud of non disaster tweets", fontsize=20)
ax2.axis("off")

fig.show()

## 7. Train the model with new training data and predict using new test data

### 7.1 Text vectorizaton

In [ ]:
count_vectorizer = CountVectorizer()
train_vectors = count_vectorizer.fit_transform(train["text"])
test_vectors = count_vectorizer.transform(test["text"])

### 7.2 Train the model

In [ ]:
clf = LogisticRegression()

scores = cross_val_score(clf, train_vectors, train["target"], cv=5, scoring="f1")
for k, score in zip(range(len(scores)),scores):
    print("F1 Score for fold %d is %.2f " % (k+1,score))
    
clf.fit(train_vectors, train["target"])

### 7.3 Make predictions and evaluate the model

In [ ]:
test["pred"] = clf.predict(test_vectors)

In [ ]:
print("We got %.1f%% accuracy on our test dataset" % (float(accuracy_score(test.target, test.pred))*100))

In [ ]:
tn, fp, fn, tp = confusion_matrix(test.target, test.pred).ravel()
tot = confusion_matrix(test.target, test.pred).sum()

print("True Negative Rate: %.1f%%" % ((tn/tot)*100))
print("False Positive Rate: %.1f%%" % ((fp/tot)*100))
print("False Negative Rate: %.1f%%" % ((fn/tot)*100))
print("True Positive Rate: %.1f%%" % ((tp/tot)*100))

## Model Summary and Next Steps

We used the simple text vectorization with text cleaning and normalization which resulted in very little improvement in accuracy from 79.6% to 79.8% that is not a huge difference. But objective here was to learn about the text normalizaton and pre-processing but single approach can not provide a significant difference in results. This depends on case to case basis and combination of different approaches based on task and data.

## 8. Train the model using TF-IDF vectorization and see if that can help in some improvement 

### 8.1 TF-IDF Vectorization

For last model we used Bag-of-words, now we will use TF-IDF vectorization approach. For documentation abut TF-IDF from sklearn package follow the below link

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
train_vectors = tfidf_vectorizer.fit_transform(train["text"])
test_vectors = tfidf_vectorizer.transform(test["text"])

### 8.2 Train the model

In [ ]:
clf = LogisticRegression()

scores = cross_val_score(clf, train_vectors, train["target"], cv=5, scoring="f1")
for k, score in zip(range(len(scores)),scores):
    print("F1 Score for fold %d is %.2f " % (k+1,score))
    
clf.fit(train_vectors, train["target"])

### 8.3 Make predictions and evaluate the model

In [ ]:
test["pred"] = clf.predict(test_vectors)

In [ ]:
print("We got %.1f%% accuracy on our test dataset" % (float(accuracy_score(test.target, test.pred))*100))

In [ ]:
tn, fp, fn, tp = confusion_matrix(test.target, test.pred).ravel()
tot = confusion_matrix(test.target, test.pred).sum()

print("True Negative Rate: %.1f%%" % ((tn/tot)*100))
print("False Positive Rate: %.1f%%" % ((fp/tot)*100))
print("False Negative Rate: %.1f%%" % ((fn/tot)*100))
print("True Positive Rate: %.1f%%" % ((tp/tot)*100))

## Tutorial Summary and Next Steps

In this tutorial notebook we tried following approaches:
    
1. Approach 1: Logistic Model with bag of words approach which resulted in 79.6% accuracy on test data.
2. Approach 2: Logistic Model using bag of words using cleaned and normalized data which resulted in 79.8% accuracy on test data.
3. Approach 3: Logistic Model using TF-IDF vectorization using cleaned and normalized data which resulted in 80.1% accuracy on test data.
   
This was introduction to Natural Language Processing where we used text data for perdiction and it was just a start. For more accuracy we might need to perform some different text normalization, use of different model instead of Logistic Regression or different metric to evaluate our model.  

Landscape of Natural language processing has changed a lot in recent years with state of art research and development in this field with use availability of huge computation power and advancment in Neaural Network Models specially Recurrent Neural Networks.